# Particle segmentation App

### If you have not already uploaded data, click on the link below data and use the Upload button:
### [Adding data](https://fl-7-206.zhdk.cloud.switch.ch)

In [2]:
import os, glob, copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import nanoparticles as nano
import skimage
from matplotlib.font_manager import FontProperties
from IPython.display import clear_output
from tqdm import tqdm_notebook

plt.style.use('ggplot')
font = FontProperties()
font.set_style('normal')
font.set_size(15)

In [19]:
path_widget = widgets.Select(options = ['None','...']+os.listdir(), value = 'None', rows = 10)
#path_widget.layout.width = '400px'

scale_widget = widgets.FloatText(step = 0.1, value=1.0, description='Scale:',disabled=False)

match_widget = widgets.FloatSlider(description = 'Quality',min = 0, max = 1, 
                                   step = 0.1, value = 0.5, continuous_update = False)

minrad_widget = widgets.IntSlider(description = 'Min radius',min = 0, max = 100, 
                                   step = 10, value = 30, continuous_update = False)

maxrad_widget = widgets.IntSlider(description = 'Max radius',min = 0, max = 100, 
                                   step = 10, value = 50, continuous_update = False)

do_widget = widgets.Dropdown(
    options=['No', 'Single', 'All'],
    value='No',
    description='Running:',
    disabled=False,
)

plotall_widget = widgets.Checkbox(description = 'Plot each image', value=False)

In [15]:
class Nano:
    
    def __init__(self):
        
        self.image = None
        self.circles = None
        self.all_radii = None
        self.plotting = False
        self.fig = None

current_nano = Nano()

In [16]:
def analyze_particles(path_to_file, min_rad, max_rad, scale, match_quality = 0.2, do_analysis = False, plot_all = False):
    """Main function analyzing particles size in EM images
    
    Parameters
    ----------
    path_to_file : str
        path to folder containing tif files
    minrad: int
        minimum radius to consider
    maxrad: int
        maximum radius to consider
    scale: float
        image scale (in nanometer per pixel)
    match_quality: float
        threshold on matching quality. 0 means all local maxima are selected
    do_analysis: bool
        Run analysis or not
    
    Returns
    -------
    all_radii: list
        list of all measured radii in all images
    
    """    
    
    old_plotting= copy.copy(current_nano.plotting)
    current_nano.plotting = copy.copy(plot_all)
    if old_plotting and not plot_all:
        fig, ax = plt.subplots(figsize=(0.1,0.1))
        ax.set_axis_off()
        plt.show()
        return
    
    if path_to_file == 'None':
        None
    elif path_to_file =='...':
        newpath = os.path.dirname(os.getcwd())
        os.chdir(newpath)
        if len(os.listdir(newpath))==0:
            path_widget.value = 'None'
            path_widget.options = ['None', '...']
        else:
            #os.chdir(newpath)
            path_widget.options = ['None','...']+os.listdir()
            path_widget.value = 'None'#os.listdir(newpath)[0]

    else:
        newpath = os.path.abspath(os.curdir)+'/'+path_to_file
        if os.path.isdir(newpath):
            os.chdir(newpath)
            if len(os.listdir(newpath))==0:
                path_widget.options = ['None', '...']
                path_widget.value = 'None'
            else:
                #os.chdir(newpath)
                path_widget.options = ['None','...']+os.listdir()
                path_widget.value = 'None'#os.listdir(newpath)[0]
                
    
    path_to_file = os.path.abspath(os.curdir)
    
    
    image = []
    circles = []
    all_radii = []
    if do_analysis != 'No':
        
        tif_files = glob.glob(path_to_file+'/*tif')
        if do_analysis == 'Single':
            tif_files = tif_files[0:1]

        all_radii = []
        for tif in tqdm_notebook(tif_files, desc='files'):
        #for tif in tif_files:
            image = skimage.io.imread(tif)
            if len(image.shape)==3:
                image = image[:,:,1]+0.001
            im_filt = nano.init_filtering(image, np.arange(min_rad, max_rad,10))
            radii, circles, intensities = nano.muli_radius_fitting(image, im_filt, min_rad, max_rad, match_quality)

            if plot_all:
                fig, ax = plt.subplots(1,2,figsize=(15,7))
                current_nano.fig  =fig
                ax[0].imshow(image, cmap = 'gray')

                if len(circles) > 0:
                    for x in circles:
                        plot_circ = plt.Circle((x[1], x[0]), x[2], color='r',linewidth = 1, faceColor = [1,1,1,0])
                        ax[0].add_artist(plot_circ)
                    ax[0].set_axis_off()

                if len(radii) > 0:
                    ax[1].hist(np.array(radii)*scale, bins = np.linspace(np.min(radii),np.max(radii),10)*scale)
                    ax[1].set_xlabel('Radius [nm]',fontproperties = font)
                    ax[1].set_ylabel('Count',fontproperties = font)
                    ax[1].tick_params(labelsize=12)

                plt.show()

            #plot_detection(image, circles, radii, scale)
            pd_temp = pd.DataFrame({'radii': radii,'intensities': intensities, 'filename': os.path.basename(tif)})
            all_radii.append(pd_temp)
        all_radii = pd.concat(all_radii)
        current_nano.all_radii = all_radii
    return image, circles, all_radii

In [23]:
out = widgets.interactive_output(analyze_particles,
                                 {'path_to_file': path_widget,
                                  'match_quality': match_widget,
                                 'min_rad': minrad_widget,
                                 'max_rad': maxrad_widget,
                                 'scale': scale_widget,
                                 'do_analysis': do_widget,
                                 'plot_all': plotall_widget})
ui = widgets.VBox([widgets.VBox([widgets.Label('Select folder containing tif files'),path_widget]), match_widget, minrad_widget, maxrad_widget,scale_widget, do_widget,plotall_widget])
                   

In [24]:
display(ui, out)

Output()

In [8]:
def plot_final(current_nano):
    clear_output()
    #plt.clf()
    display(final_button)
    fig, ax = plt.subplots()
    plt.hist(current_nano.all_radii.radii*scale_widget.value, 
             bins = np.linspace(current_nano.all_radii.radii.min(),current_nano.all_radii.radii.max(),10)*scale_widget.value)
    ax.set_xlabel('Radius [nm]',fontproperties = font)
    ax.set_ylabel('Count',fontproperties = font)
    ax.tick_params(labelsize=12)

def on_click_action(b):
    plot_final(current_nano)
    

In [9]:
final_button = widgets.Button(description = 'Plot final')
final_button.on_click(on_click_action)

In [10]:
final_button

Button(description='Plot final', style=ButtonStyle())